# Task 1: Define language

In [1]:
import pandas as pd, numpy as np

Reading files and detecting given languages manually.

Reading 2.tmp

In [2]:
dataset1 = pd.read_csv('2.tmp', names=['text'], sep='\n').dropna()
dataset1[:1]

,text
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...


In [3]:
dataset1['lang'] = 'chinese'

Reading 3.tmp

In [4]:
dataset2 = pd.read_csv('3.tmp', names=['text'], sep='\n').dropna()
dataset2[:1]

,text
0,When Mr. Bilbo Baggins of Bag End announced th...


In [5]:
dataset2['lang'] = 'english'

Reading 4.tmp

In [6]:
dataset3 = pd.read_csv('4.tmp', names=['text'], sep='\n').dropna()
dataset3[:1]

,text
0,Bag EndのBilbo Baggins氏は、18歳の誕生日を特別な壮大なパーティでまもな...


In [7]:
dataset3['lang'] = 'japanese'

Joining all datasets

In [8]:
dataset = pd.concat((dataset1, dataset2, dataset3))
dataset.index = np.arange(len(dataset))

Understanding dataset representation:

In [9]:
dataset.head()

,text,lang
0,Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...,chinese
1,什麼，人們普遍相信，Bag End山充滿了寶藏的隧道。如果這還不夠名聲，那麼他也有長期的活力...,chinese
2,青年以及（著名地）無窮無盡的財富似乎是不公平的。他們說：“這將需要支付。” “這不是自然的，...,chinese
3,仍然與親屬（除了當然是薩克維爾 - 巴金斯）的訪問條件，他在貧窮和不重要的家庭的愛好中有許多...,chinese
4,＃＃”你說得對，爸爸！“Gaffer說。 “不是巴剋土地的白蘭地酒在老森林裡生活，但他們似乎...,chinese


Understanding hex representation

In [10]:
hex(ord('日'))

'0x65e5'

Then implementing classifiers

In [11]:
def classifySymbol(c):
    # 0x0020—0x007F, 0x2000—0x206F - english
    def isEnglish(c):
        return int('0x0020', base=16) <= ord(c) <= int('0x007F', base=16) or \
               int('0x2000', base=16) <= ord(c) <= int('0x206F', base=16)
    
    # 0x3040—0x30FF - japanese
    def isJapanese(c):
        return int('0x3040', base=16) <= ord(c) <= int('0x30FF', base=16)
    
    # 0x4E00—0x9FFF - chineese
    def isChinese(c):
        return int('0x4E00', base=16) <= ord(c) <= int('0x9FFF', base=16)

    
    if isEnglish(c):
        return 'english'
    elif isJapanese(c):
        return 'japanese'
    elif isChinese(c):
        return 'chinese'
    else:
        return 'undef'

In [12]:
def classifyString(line):
    r = dict({'english': 0, 'japanese': 0, 'chinese': 0, 'undef': 0})
    for c in line:
        r[classifySymbol(c)] += 1
    return r

Small test

In [13]:
classifyString('lalaka日別的輝煌的聚壮大なパーティでまも')

{'chinese': 9, 'english': 6, 'japanese': 8, 'undef': 0}

Using KFold because required

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [15]:
kf = KFold(n_splits=5, shuffle=True)

In [16]:
dataset['text'][:1].to_string(index=False)

'Bag End先生的Bilbo Baggins先生宣布，他不久將以特別的輝煌的聚會慶祝他的第...'

In [17]:
for train_index, test_index in kf.split(dataset):
    
    train_texts = np.array(dataset['text'])[train_index]
    test_texts = np.array(dataset['text'])[test_index]

    true_train = np.array(dataset['lang'])[train_index]
    true_test = np.array(dataset['lang'])[test_index]
    
    pred_train = [max(d, key=d.get) for d in [classifyString(s) for s in train_texts]]
    pred_test = [max(d, key=d.get) for d in [classifyString(s) for s in test_texts]]
    
    print(accuracy_score(true_train, pred_train))
    print(accuracy_score(true_test, pred_test))

0.9903474903474904
0.987146529562982
0.9897039897039897
0.9897172236503856
0.9903474903474904
0.987146529562982
0.9897106109324759
0.9896907216494846
0.9884244372990354
0.9948453608247423


### Trying to modify classifier

1. Управляющие символы(0x0000-0x001F). В нашем случае их всего два: нуль-символ и символ перевода строки. Они не влияют на классификацию, поэтому их можно не учитывать.
2. Основная латинница(0x0020—0x007F). Содержит также графические знаки. В нашем датасете соответствует лишь английскому языку, однако арабские цифры (в силу присутствия их во всех текстах) мы будем относить ко всем трем языкам.
3. Знаки пунктуации(0x2000—0x206F). Будем относить к английскому языку, так как для языков восточной группы существуют свои знаки препинания.
4. Символы и пунктуация ККЯ(0x3000—0x303F). Используются в китайском и японском языках.
5. Катакана и хирагана(0x3040—0x30FF). Используются в японском языке.
6. Унифицированные иероглифы ККЯ(0x4E00—0x9FFF). Используются в основном в китайском языке.
7. Полуширинные и полноширинные формы(0xFF00—0xFFEF). Используются в китайском и японском языках.

#### @copyright 
#### Источник классификации символов: Сергей Винокуров. (заимствован только текст этой клетки)

In [18]:
def classifySymbol(c):
    def isEnglish(c):
        return 0x0020 <= ord(c) <= 0x007F or \
               0x2000 <= ord(c) <= 0x206F
        
    
    def isJapanese(c):
        return 0x3040 <= ord(c) <= 0x30FF or \
               0x3000 <= ord(c) <= 0x303F or \
               0xFF00 <= ord(c) <= 0xFFEF
    
    def isChinese(c):
        return 0x4E00 <= ord(c) <= 0x9FFF or \
               0x3000 <= ord(c) <= 0x303F or \
               0xFF00 <= ord(c) <= 0xFFEF

    res = dict({'english': 0, 'japanese': 0, 'chinese': 0, 'undef': 0})
    
    if isEnglish(c):
        res['english'] = 1
    elif isJapanese(c):
        res['japanese'] = 1
    elif isChinese(c):
        res['chinese'] = 1
    else:
        res['undef'] = 1
    return res

In [19]:
def classifyString(line):
    r = dict({'english': 0, 'japanese': 0, 'chinese': 0, 'undef': 0})
    for c in line:
        for k,v in classifySymbol(c).items(): 
            r[k] += v
    return r

Test

In [20]:
classifyString('asdfasdf')

{'chinese': 0, 'english': 8, 'japanese': 0, 'undef': 0}

In [21]:
for train_index, test_index in kf.split(dataset):
    
    train_texts = np.array(dataset['text'])[train_index]
    test_texts = np.array(dataset['text'])[test_index]

    true_train = np.array(dataset['lang'])[train_index]
    true_test = np.array(dataset['lang'])[test_index]
    
    pred_train = [max(d, key=d.get) for d in [classifyString(s) for s in train_texts]]
    pred_test = [max(d, key=d.get) for d in [classifyString(s) for s in test_texts]]
    
    print(accuracy_score(true_train, pred_train))
    print(accuracy_score(true_test, pred_test))

0.9916344916344917
0.9948586118251928
0.9929214929214929
0.9897172236503856
0.990990990990991
0.9974293059125964
0.9929260450160772
0.9896907216494846
0.9929260450160772
0.9896907216494846


### Nice results. Let's try something else

In [22]:
from langdetect import detect
def classifyString_1(line):
    res = dict()
    try:
        key = detect(line)[:2]
    except:
        return 'english'
    
    if (key == 'zh'):
        key = 'chinese'
    elif (key == 'ja'):
        key = 'japanese'
    else:
        key = 'english'
    
    return key

In [23]:
for train_index, test_index in kf.split(dataset):
    
    train_texts = np.array(dataset['text'])[train_index]
    test_texts = np.array(dataset['text'])[test_index]

    true_train = np.array(dataset['lang'])[train_index]
    true_test = np.array(dataset['lang'])[test_index]
    
    pred_train = [classifyString_1(s) for s in train_texts]
    pred_test = [classifyString_1(s) for s in test_texts]
    
    print(accuracy_score(true_train, pred_train))
    print(accuracy_score(true_test, pred_test))

0.9626769626769627
0.9485861182519281
0.9658944658944659
0.9537275064267352
0.9613899613899614
0.9588688946015425
0.9588424437299036
0.9510309278350515
0.9594855305466238
0.9664948453608248


### Quite nice result too.